### Melon 100 챠트 스크래핑
1. 100곡의 노래의 제목과 ID 추출 (정규표현식)
2. 노래의 ID로 상세페이지를 100번 요청해서 상세정보를 추출
3. Json file에 저장
4. json file을 DataFrame 객체로 생성해서 표데이터 만들기
5. 표데이터를 DB에 저장하기

In [22]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
# 요청헤더 
req_header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
}
res = requests.get(url, headers=req_header)
print(res.status_code)

if res.ok:
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    # <div id="tb_list"> 
    print(len(soup.select('div#tb_list'))) #1
    # <div id="tb_list">  하위의 tr 선택
    print(len(soup.select('div#tb_list tr'))) #101
    # <div id="tb_list">  하위의 tr 하위의 <div class="wrap_song_info">
    print(len(soup.select('div#tb_list tr div.wrap_song_info'))) #200
    # <div id="tb_list">  하위의 tr 하위의 <div class="wrap_song_info"> 하위의 <a href="playSong">
    print(len(soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']"))) #100
    
    song_atag = soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']")
    # 100개의 song정보를 저장할 리스트 선언
    song_list = []
    for song in song_atag:
        # 1개의 song 정보를 저장한 딕셔너리 선언
        song_dict = {}
        title = song.text
        href_value = song['href']
        matched = re.search(r'(\d+)\);', href_value)
        #print(matched.group(0), matched.group(1))
        song_id = matched.group(1)
        song_detail_link = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        song_dict['title'] = title
        song_dict['link'] = song_detail_link
        song_list.append(song_dict)

print(len(song_list))
song_list[0:5]

200
1
101
200
100
100


[{'title': "롤린 (Rollin')",
  'link': 'https://www.melon.com/song/detail.htm?songId=30287019'},
 {'title': '라일락',
  'link': 'https://www.melon.com/song/detail.htm?songId=33372781'},
 {'title': 'Peaches (Feat. Daniel Caesar & Giveon)',
  'link': 'https://www.melon.com/song/detail.htm?songId=33359317'},
 {'title': 'Celebrity',
  'link': 'https://www.melon.com/song/detail.htm?songId=33239419'},
 {'title': '운전만해 (We Ride)',
  'link': 'https://www.melon.com/song/detail.htm?songId=32860397'}]

In [23]:
import requests
from bs4 import BeautifulSoup
import re

# 요청헤더 
req_header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
}
song_lyric_lists = []
song_lyric_dict = {}

for idx,song in enumerate(song_list,1):
    html = requests.get(song['link'], headers=req_header).text
    soup = BeautifulSoup(html,'html.parser')
    song_lyric_dict['곡명'] = song['title']
    song_lyric_dict['가수'] = soup.select('a[href*=".goArtistDetail"] span')[0].text
    song_lyric_dict['발매일'] = soup.select('div.meta dd')[1].text
    song_lyric_dict['장르'] = soup.select('div.meta dd')[2].text
    print(idx, song['link'])
#    print(soup.select('div#d_video_summary')[0].text)
    if(soup.select('div#d_video_summary')):
        lyric = soup.select('div#d_video_summary')[0].text
    else:
        lyric = ''
    regex = re.compile(r'[\n\r\t]')
    song_lyric_dict['가사'] = regex.sub('', lyric.strip())
    song_lyric_lists.append(song_lyric_dict)
    song_lyric_dict = {}

print(len(song_lyric_lists)) 

1 https://www.melon.com/song/detail.htm?songId=30287019
2 https://www.melon.com/song/detail.htm?songId=33372781
3 https://www.melon.com/song/detail.htm?songId=33359317
4 https://www.melon.com/song/detail.htm?songId=33239419
5 https://www.melon.com/song/detail.htm?songId=32860397
6 https://www.melon.com/song/detail.htm?songId=1944399
7 https://www.melon.com/song/detail.htm?songId=1854856
8 https://www.melon.com/song/detail.htm?songId=1500196
9 https://www.melon.com/song/detail.htm?songId=33372783
10 https://www.melon.com/song/detail.htm?songId=3414749
11 https://www.melon.com/song/detail.htm?songId=32872978
12 https://www.melon.com/song/detail.htm?songId=33346446
13 https://www.melon.com/song/detail.htm?songId=33331004
14 https://www.melon.com/song/detail.htm?songId=33167063
15 https://www.melon.com/song/detail.htm?songId=33061995
16 https://www.melon.com/song/detail.htm?songId=33405149
17 https://www.melon.com/song/detail.htm?songId=711626
18 https://www.melon.com/song/detail.htm?songI

In [25]:
song_lyric_lists[0:2]

[{'곡명': "롤린 (Rollin')",
  '가수': '브레이브걸스',
  '발매일': '2017.03.07',
  '장르': '댄스',
  '가사': "그 날을 잊지 못해 babe날 보며 환히 웃던 너의 미소에홀린 듯 I'm fall in loveBut 너무 쪽팔림에 난 그저한마디 말도 못해 babe I wanna you 너의 눈빛은 날 자꾸 네 곁을 맴돌게 해Just only you 굳게 닫힌 내 맘이 어느새 무너져버려 Because of you온통 너의 생각뿐이야 나도 미치겠어너무 보고 싶어 매일 매일 매일자꾸 초라해지잖아 내 모습이그대여 내게 말해줘 사랑한다고Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' 하루가 멀다 하고 Rolling in the deep Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' 기다리고 있잖아 Babe Just only you기다리고 있잖아 Babe Just only youHey I just wanna be with you오늘 밤이 가기 전에I can't feel you 조금 더 다가와 줘 Tonight I'm ready for youYou wanna touch me I know대체 뭘 고민해 빨리 안아아닌 척 모르는 척 하다가늦게 놓치고 후회 말아I wanna you 너의 눈빛은 날 자꾸 네 곁을 맴돌게 해Just only you 굳게 닫힌 내 맘이 어느새 무너져버려 Because of you온통 너의 생각뿐이야 나도 미치겠어너무 보고 싶어 매일 매일 매일자꾸 초라해지잖아 내 모습이그대여 내게 말해줘 사랑한다고Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' Rollin' 하루가 멀다 하고 Rolling in the deep R

In [27]:
import json

with open('data/songs.json','w', encoding='utf-8') as file:
    json.dump(song_lyric_lists, file)

In [28]:
import pandas as pd

song_df = pd.read